<a href="https://colab.research.google.com/github/li0217codeninja/time-seq-learning/blob/main/timeseriesTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.10.12


In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

## Timeseries PyTorch Transformer

In [ ]:
class SequenceDataset(torch.utils.data.Dataset):
    def __init__(self, seq_length):
        self.seq_length = seq_length

    def __len__(self):
        return 10000  # Replace with actual dataset size

    def __getitem__(self, idx):
        # Generate a sequence number
        seq_num = torch.tensor(idx)

        # Generate a sequence of numbers based on seq_num
        seq_data = torch.randn(self.seq_length) * seq_num

        return seq_num, seq_data

def collate_fn(samples):
    seq_nums, seq_data = zip(*samples)
    # Pad or truncate sequences to a fixed length (optional)
    # ...
    return torch.stack(seq_nums), torch.stack(seq_data)

train_loader = DataLoader(
    SequenceDataset(seq_length=32),
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn,
)


In [ ]:
transformer_model = nn.Transformer(nhead=16)
src = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
out = transformer_model(src, tgt)

In [ ]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(transformer_model.parameters())

epochs = 10
for epoch in range(epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    # Forward pass
    output = transformer_model(data,target)
    # Loss calculation
    loss = loss_fn(output, target)
    # Backward pass and parameter update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
transformer_model.parameters()

<generator object Module.parameters at 0x7a4ca5ac67a0>

## Custom train loop with LSTM

1.   understand the bi-direction argument in lstm
2.   how to dignose if any issue with model trainining
3.   one step prediction, can we do multi-step forcast? is it just a for loop.





In [4]:
class RNNmodel(nn.Module):
  def __init__(self, input_size, hidden_size,  num_features):
    super(RNNmodel,self).__init__() #todo check super()
    self.lstm = nn.LSTM(input_size, hidden_size,  batch_first=True)
    self.dense = nn.Linear(hidden_size, num_features)

  def forward(self, x):
    h, _ = self.lstm(x) # output, (h_n, c_n)
    output = self.dense(h[:,-1,:])
    return output


In [5]:
class DummyDataset(Dataset):
  def __init__(self,  data, sequence_len):
    # 100 sequences, 10 time steps, 2 features per variable, 3 variables
    self.data = data
    self.sequence_length = sequence_len

  def __len__(self):
    return len(self.data) - self.sequence_length

  def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx + self.sequence_length, :], dtype=torch.float32)
        y = torch.tensor(self.data[idx + self.sequence_length, :], dtype=torch.float32)
        return x, y

In [39]:
# Generate dummy data
np.random.seed(42)
num_samples = 1000
sequence_length = 10
num_features = 3

data = np.random.randn(num_samples, num_features)

# Create the model, dataset and data loader
input_size = num_features
hidden_size = 50
output_size = num_features

model = RNNmodel(input_size, hidden_size, output_size)
dataset = DummyDataset(data, sequence_length)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(100):
  for batch_x, batch_y in dataloader:
        # Training logic goes here
        # e.g., forward pass, loss computation, backward pass, and optimization
        outputs = model(batch_x)
        loss = nn.MSELoss()(outputs, batch_y)

        # Your training steps here
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  print(f'Epoch {epoch + 1}, Batch Loss: {loss.item()}')



Epoch 1, Batch Loss: 0.7638987898826599
Epoch 2, Batch Loss: 1.0971240997314453
Epoch 3, Batch Loss: 0.8734275698661804
Epoch 4, Batch Loss: 1.013189673423767
Epoch 5, Batch Loss: 0.7677677273750305
Epoch 6, Batch Loss: 1.1158329248428345
Epoch 7, Batch Loss: 0.929412305355072
Epoch 8, Batch Loss: 1.2866199016571045
Epoch 9, Batch Loss: 0.9599927067756653
Epoch 10, Batch Loss: 1.1041592359542847
Epoch 11, Batch Loss: 0.9852710366249084
Epoch 12, Batch Loss: 1.0059996843338013
Epoch 13, Batch Loss: 0.9701211452484131
Epoch 14, Batch Loss: 0.9712355136871338
Epoch 15, Batch Loss: 1.078649878501892
Epoch 16, Batch Loss: 0.9533963203430176
Epoch 17, Batch Loss: 0.7716512084007263
Epoch 18, Batch Loss: 1.06806480884552
Epoch 19, Batch Loss: 1.1129637956619263
Epoch 20, Batch Loss: 0.9060471057891846
Epoch 21, Batch Loss: 0.9639253616333008
Epoch 22, Batch Loss: 1.0287281274795532
Epoch 23, Batch Loss: 0.8987563252449036
Epoch 24, Batch Loss: 0.9339909553527832
Epoch 25, Batch Loss: 1.179357

In [37]:
# Assuming model is your PyTorch model
initial_params = sum(p.numel() for p in model.parameters())
print("Initial number of parameters:", initial_params)

# Check if any parameters have been updated
updated_params = sum(p.numel() for p in model.parameters())
if updated_params > initial_params:
    print("The model has been trained.")
else:
    print("The model has not been trained.")

Initial number of parameters: 11153
The model has not been trained.


In [35]:
# Inference
# After training, you can use the trained model for predictions
test_sequence = torch.tensor(np.random.randn(11, num_features), dtype=torch.float32).unsqueeze(0)  # Add batch dimension
test_sequence
prediction = model(test_sequence)

mse = nn.MSELoss()(prediction, test_sequence[:,-1,:])
mse

tensor(0.9112, grad_fn=<MseLossBackward0>)

In [44]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

print()

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
lstm.weight_ih_l0 	 torch.Size([200, 3])
lstm.weight_hh_l0 	 torch.Size([200, 50])
lstm.bias_ih_l0 	 torch.Size([200])
lstm.bias_hh_l0 	 torch.Size([200])
dense.weight 	 torch.Size([3, 50])
dense.bias 	 torch.Size([3])

Optimizer's state_dict:
state 	 {0: {'momentum_buffer': tensor([[ 1.9207e-03,  1.3583e-03,  9.2337e-04],
        [ 4.1726e-05,  1.1890e-03,  4.0827e-05],
        [-7.4290e-04, -1.2806e-04, -2.7775e-05],
        [-2.4266e-04, -8.6451e-04, -5.1736e-04],
        [ 1.5325e-04, -2.6185e-03, -9.1021e-04],
        [-1.6743e-03,  2.5834e-03,  8.1540e-04],
        [ 6.8749e-04, -3.5512e-03,  3.7221e-04],
        [ 1.8752e-04, -1.8760e-03,  6.6700e-04],
        [-9.9014e-04,  9.0225e-05,  1.8158e-04],
        [ 4.7678e-04, -1.4619e-03, -6.3338e-04],
        [ 3.9095e-04, -1.0312e-03,  2.9193e-04],
        [-7.3947e-04,  1.3365e-03,  1.4538e-03],
        [ 1.0970e-03, -3.9511e-05,  4.9406e-04],
        [ 4.4125e-04, -7.3113e-04,  2.3710e-04],
        [ 5.2031e-